In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import RandomSampler
from torch.autograd import Variable
from torchvision import transforms
import matplotlib.pyplot as plt
from torchvision.transforms.functional import normalize
#from google.colab import drive                # IF you are using COLAB

In [ ]:
# Use this if you are working on COLAB
# This will prompt for authorization.
#drive.mount('/content/drive')

In [ ]:
def extract_data(x_data_filepath, y_data_filepath):
    X = np.load(x_data_filepath)
    y = np.load(y_data_filepath)
    return X, y

In [ ]:
def data_visualization(images,labels):
    """
    Visualize 6 pictures per class using your prefered visualization library (matplotlib, etc)

    Args:
        images: training images in shape (num_images,3,image_H,image_W)
        labels: training labels in shape (num_images,)
    """
    # Extract the indices of 6 images of each class from labels
    lst_labels = [int(x) for x in labels]
    label_0 = [index for index, value in enumerate(lst_labels) if value == 0][:6]
    label_1 = [index for index, value in enumerate(lst_labels) if value == 1][:6]
    label_2 = [index for index, value in enumerate(lst_labels) if value == 2][:6]
    label_3 = [index for index, value in enumerate(lst_labels) if value == 3][:6]
    label_4 = [index for index, value in enumerate(lst_labels) if value == 4][:6]
    y = [label_0,label_1, label_2, label_3, label_4]
    for i in range(5):
        for j in y[i]:
            plt.figure()
            plt.imshow(images[j][:,:,:].T.swapaxes(0,1))

In [ ]:
############################################################
# Extracting and loading data
############################################################
class Dataset(Dataset):
    def __init__(self, X, y):
        self.len = len(X)           
        if torch.cuda.is_available():
            self.x_data = torch.from_numpy(X).float().cuda()
            self.y_data = torch.from_numpy(y).long().cuda()
        else:
            self.x_data = torch.from_numpy(X).float()
            self.y_data = torch.from_numpy(y).long()
    
    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]

In [ ]:
def create_validation(x_train,y_train):
    """
    Randomly choose 20 percent of the training data as validation data.

    Args:
        x_train: training images in shape (num_images,3,image_H,image_W)
        y_train: training labels in shape (num_images,)
    Returns:
        new_x_train: training images in shape (0.8*num_images,3,image_H,image_W)
        new_y_train: training labels in shape (0.8*num_images,)
        x_val: validation images in shape (0.2*num_images,3,image_H,image_W)
        y_val: validation labels in shape (0.2*num_images,)
    """
    data = Dataset(x_train,y_train)
    train_size = int(np.floor(0.8 * len(data)))
    indices= list(range(len(data)))
    valid_size = len(data) - train_size
    np.random.shuffle(indices)
    train_mapping=indices[valid_size:]
    valid_mapping=indices[:valid_size]
    new_x_train = data.x_data[train_mapping]
    x_val = data.x_data[valid_mapping]
    new_y_train = data.y_data[train_mapping]
    y_val = data.y_data[valid_mapping]
    
    return new_x_train,new_y_train,x_val,y_val

In [ ]:
############################################################
# Feed Forward Neural Network
############################################################
class FeedForwardNN(nn.Module):
    """ 
        (1) Use self.fc1 as the variable name for your first fully connected layer
        (2) Use self.fc2 as the variable name for your second fully connected layer
    """
    def __init__(self):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(16320,2000)
        self.fc2 = nn.Linear(2000,5)
    
    def forward(self, x):
        # Reshaping the input to 16320*1
        x_flatten = x.view(x.size(0),-1)
        # Computing the output of the first feedforward layer
        out1 = F.relu(self.fc1(x_flatten))
        # Computing the output of the second feedforward layer
        out = self.fc2(out1)
        return out
        
    """ 
        Please do not change the functions below. 
        They will be used to test the correctness of your implementation 
    """
    def get_fc1_params(self):
        return self.fc1.__repr__()
    
    def get_fc2_params(self):
        return self.fc2.__repr__()

In [ ]:
############################################################
# Convolutional Neural Network
############################################################
class ConvolutionalNN(nn.Module):
    """ 
        (1) Use self.conv1 as the variable name for your first convolutional layer
        (2) Use self.pool1 as the variable name for your first pooling layer
        (3) Use self.conv2 as the variable name for your second convolutional layer
        (4) Use self.pool2 as the variable name for you second pooling layer  
        (5) Use self.fc1 as the variable name for your first fully connected laye
        (6) Use self.fc2 as the variable name for your second fully connected layer
    """
    def __init__(self):
        super(ConvolutionalNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size = 3,stride=1, padding=0)
        self.pool1 = nn.MaxPool2d(kernel_size = 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size = 3,stride=1, padding=0)
        self.conv3 = nn.Conv2d(32, 64, kernel_size = 3,stride=1, padding=0)
        self.pool3 =nn.MaxPool2d(kernel_size = 2)
        self.pool2 = nn.MaxPool2d(kernel_size = 2)
        self.fc1 = nn.Linear(3072, 200)
        self.fc2 = nn.Linear(200, 5)
        self.dropout = nn.Dropout(p=0.5)
        self.dropout1 = nn.Dropout(p=0.2)
    
    def forward(self, x):
        # Normalisation
        x_numpy = x.cpu().numpy()
        x = normalize_image(x_numpy)
        out = F.relu(self.conv1(x))
        out = self.pool1(out)
        out = F.relu(self.conv2(out))
        out = self.pool2(out)
        out = F.relu(self.conv3(out))
        out = self.pool3(out)
        out = out.view(out.size(0),-1)
        out = self.dropout(out)
        out = F.relu(self.fc1(out))
        out = self.dropout1(out)
        out = self.fc2(out)
        return out
      
    """ 
        Please do not change the functions below. 
        They will be used to test the correctness of your implementation
    """
    
    def get_conv1_params(self):
        return self.conv1.__repr__()
    
    def get_pool1_params(self):
        return self.pool1.__repr__()

    def get_conv2_params(self):
        return self.conv2.__repr__()
      
    def get_pool2_params(self):
        return self.pool2.__repr__()
      
    def get_fc1_params(self):
        return self.fc1.__repr__()
    
    def get_fc2_params(self):
        return self.fc2.__repr__()

In [ ]:
def normalize_image(image):
    """
    Normalize each input image

    Args:
        image: the input image in shape (3,image_H,image_W)
    Returns:
        norimg: the normalized image in the same shape as the input
    """
    np_x = image.copy()
    # Mean per channel
    red_mean = np.mean(np_x[0,:,:])
    green_mean = np.mean(np_x[1,:,:])
    blue_mean = np.mean(np_x[2,:,:])
    # Standard Deviation per channel
    red_std = np.std(np_x[0,:,:])
    green_std = np.std(np_x[1,:,:])
    blue_std = np.std(np_x[2,:,:])
    # normalize in tensor
    np_x[0,:,:] = (np_x[0,:,:] - red_mean)/red_std
    np_x[1,:,:] = (np_x[1,:,:] - green_mean)/green_std
    np_x[2,:,:] = (np_x[2,:,:] - blue_mean)/blue_std
    norimg = torch.tensor(np_x)
    return norimg.cuda()

In [ ]:
############################################################
# Optimized Neural Network
############################################################
class OptimizedNN(nn.Module):
    def __init__(self):
        super(OptimizedNN, self).__init__()
        self.fc1 = nn.Linear(16320,10000) 
        self.fc2 = nn.Linear(10000,5000)
        self.fc3 = nn.Linear(5000,2048)
        self.fc4 = nn.Linear(2048,1024)
        self.fc5 = nn.Linear(1024,512)
        self.fc6 = nn.Linear(512,256)
        self.fc7 = nn.Linear(256,124)
        self.fc8 = nn.Linear(124,5)
    def forward(self, x):
        # Normalize the image
        x_numpy = x.cpu().numpy()
        x = normalize_image(x_numpy)
        # Reshaping the input to 16320*1
        x_flatten = x.view(x.size(0),-1)
        out = F.relu(self.fc1(x_flatten))
        out = F.relu(self.fc2(out))
        out = F.relu(self.fc3(out))
        out = F.relu(self.fc4(out))
        out = F.relu(self.fc5(out))
        out = F.relu(self.fc6(out))
        out = self.fc7(out)
        return out
      

In [ ]:
def train_val_NN(neural_network, train_loader, validation_loader, loss_function, optimizer,num_epochs):
    """
    Runs experiment on the model neural network given a train loader, loss function and optimizer and find validation 
    accuracy for each epoch given the validation_loader.

    Args:
        neural_network (NN model that extends torch.nn.Module): For example, it should take an instance of either
                                                                FeedForwardNN or ConvolutionalNN,
        train_loader (DataLoader),
        validation_loader (DataLoader),
        loss_function (torch.nn.CrossEntropyLoss),
        optimizer (optim.SGD)
        num_epochs (number of iterations)
    Returns:
        tuple: First position, training accuracies of each epoch formatted in an array of shape (num_epochs,1).
               Second position, training loss of each epoch formatted in an array of shape (num_epochs,1).
               third position, validation accuracy of each epoch formatted in an array of shape (num_epochs,1).
               
    """
    accuracy = np.zeros(shape=(num_epochs,1))
    val_accuracy = np.zeros(shape=(num_epochs,1))
    loss_np = np.zeros(shape=(num_epochs,1))
    iterator = 0
    for epoch in range(1, num_epochs+1): ## run the model for 10 epochs
        train_loss = []
        ## training part 
        neural_network.train()
        correct = 0
        acc = 0
        val_acc = 0
        for data, target in train_loader:
            optimizer.zero_grad()
            ## forward propagation
            output = neural_network(data)
            ## loss calculation
            loss = loss_function(output, target)
            ## backward propagation
            loss.backward()
            ## weight optimization
            optimizer.step()
            train_loss.append(loss.item())
            ## accuracy computation for each batch
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
        acc = correct.item()/len(train_loader.dataset)
        ## evaluation part 
        neural_network.eval()
        correct = 0
        for data, target in validation_loader:
            output = neural_network(data)
            loss = loss_function(output, target)
            ## accuracy computation on validation set
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
        val_acc=correct.item()/len(validation_loader.dataset)   
        # Add values for each epoch to the arrays
        accuracy[iterator] = acc
        val_accuracy[iterator] = val_acc
        loss_np[iterator] = np.mean(train_loss)
        iterator+=1
    torch.cuda.empty_cache()
    return accuracy,loss_np,val_accuracy

In [ ]:
def test_NN(neural_network, test_loader):
  
    """
    Runs experiment on the model neural network given a test loader, loss function and optimizer.

    Args:
        neural_network (NN model that extends torch.nn.Module): For example, it should take an instance of either
                                                                FeedForwardNN or ConvolutionalNN,
        test_loader (DataLoader), (make sure the loader is not shuffled)
    Returns:
        your predictions         
    """
    neural_network.eval()
    Preds = torch.LongTensor()
    
    for _, data in enumerate(test_loader):
        data = data[0]
        output = neural_network(data)
        
        pred = output.data.max(1, keepdim=True)[1]
        Preds = torch.cat((Preds, pred), dim=0)
        
    return Preds

In [ ]:
# # Run Baseline FeedForward
# ############################################# Data Preprocessing #################################################

images, labels = extract_data('data\images_train.npy', 'data\labels_train.npy')
x_train, y_train, x_val, y_val = create_validation(images,labels)
train_dataset = torch.utils.data.TensorDataset(x_train,y_train)
validation_dataset = torch.utils.data.TensorDataset(x_val,y_val)
train_loader = DataLoader(train_dataset, batch_size=64)
validation_loader = DataLoader(validation_dataset, batch_size=64)

############################################# Network Objects ######################################################

num_epochs = 40
ff_nn = FeedForwardNN()
ff_nn.cuda()
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
criterion.cuda()
optimizer = optim.Adagrad(ff_nn.parameters(), lr=0.001)

accuracy,loss_np,val_accuracy = train_val_NN(ff_nn, train_loader, validation_loader, criterion, optimizer,num_epochs)

In [ ]:
np.mean(accuracy)

In [ ]:
# Run Baseline CNN
############################################# Network Objects ######################################################

num_epochs = 40
c_nn = ConvolutionalNN()
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(c_nn.parameters(), lr=0.001)

accuracy,loss_np,val_accuracy = train_val_NN(c_nn, train_loader, validation_loader, criterion, optimizer,num_epochs)

In [ ]:
# # Run Baseline CNN on Normilized Images

# images, labels = extract_data('data\images_train.npy', 'data\labels_train.npy')
# x_train, y_train, x_val, y_val = create_validation(images,labels)
# train_dataset = torch.utils.data.TensorDataset(x_train,y_train)
# validation_dataset = torch.utils.data.TensorDataset(x_val,y_val)
# train_loader = DataLoader(train_dataset, batch_size=64)
# validation_loader = DataLoader(validation_dataset, batch_size=64)

# ############################################# Network Objects ######################################################

# num_epochs = 40
# c_nn = ConvolutionalNN()
# # Loss and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adagrad(c_nn.parameters(), lr=0.001)

# accuracy,loss_np,val_accuracy = train_val_NN(c_nn, train_loader, validation_loader, criterion, optimizer,num_epochs)

In [ ]:
# Choose from one of the above models and improve its performance
torch.cuda.empty_cache()
images, labels = extract_data('data\images_train.npy', 'data\labels_train.npy')
x_train, y_train, x_val, y_val = create_validation(images,labels)
train_dataset = torch.utils.data.TensorDataset(x_train,y_train)
validation_dataset = torch.utils.data.TensorDataset(x_val,y_val)
train_loader = DataLoader(train_dataset, batch_size=64)
validation_loader = DataLoader(validation_dataset, batch_size=64)
num_epochs = 40
optimized_nn = OptimizedNN()
optimized_nn = optimized_nn.cuda()
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(optimized_nn.parameters(), lr=1e-4)
# def init_normal(m):
#     if type(m) == nn.Linear:
#         nn.init.xavier_uniform_(m.weight)
# optimized_nn = optimized_nn.apply(init_normal)
accuracy, loss_np, val_accuracy = train_val_NN(optimized_nn,train_loader, validation_loader, criterion, optimizer,num_epochs)


In [ ]:
print(accuracy[-1])
print(val_accuracy[-1])

In [ ]:
######################################### Accuracy Plot for Feedforward NN ##################################################

# lst_acc = [accuracy[i].tolist()[0] for i in range(len(accuracy))]
# epochs = [i+1 for i in range(num_epochs)]
# f, ax = plt.subplots(1)
# ax.plot(epochs, lst_acc,marker = 'o')
# ax.set_ylim(bottom=0)
# plt.title('Plot showing the variations in accuracy for training set in the Baseline Feedforward NN over 40 epochs \n')
# plt.xlabel('Epochs')
# plt.ylabel('Magnitude')
# plt.legend(['accuracy'])
# plt.show(f)

In [ ]:
# ######################################### Loss Plot for Feedforward NN ##################################################

# lst_loss = [loss_np[i].tolist()[0] for i in range(len(accuracy))]
# epochs = [i+1 for i in range(num_epochs)]
# f, ax = plt.subplots(1)
# ax.plot(epochs, lst_loss, 'r', marker = 'o',)
# ax.set_ylim(bottom=0)
# plt.title('Plot showing the variations in loss for training set in the Baseline Feedforward NN over 40 epochs \n')
# plt.xlabel('Epochs')
# plt.ylabel('Magnitude')
# plt.legend(['loss'])
# plt.show(f)

In [ ]:
######################################### Accuracy Plot for Convolutional NN ##############################################

# lst_acc = [accuracy[i].tolist()[0] for i in range(len(accuracy))]
# epochs = [i+1 for i in range(num_epochs)]
# f, ax = plt.subplots(1)
# ax.plot(epochs, lst_acc,marker = 'o')
# ax.set_ylim(bottom=0)
# plt.title('Plot showing the variations in accuracy for training set in the Convolutional NN over 40 epochs \n')
# plt.xlabel('Epochs')
# plt.ylabel('Magnitude')
# plt.legend(['accuracy'])
# plt.show(f)

In [ ]:
######################################### Loss Plot for Convolutional NN ##############################################

# lst_loss = [loss_np[i].tolist()[0] for i in range(len(accuracy))]
# epochs = [i+1 for i in range(num_epochs)]
# f, ax = plt.subplots(1)
# ax.plot(epochs, lst_loss, 'r', marker = 'o',)
# ax.set_ylim(bottom=0)
# plt.title('Plot showing the variations in loss for training set in the Convolutional NN working on\
#            normalized images over 40 epochs \n')
# plt.xlabel('Epochs')
# plt.ylabel('Magnitude')
# plt.legend(['loss'])
# plt.show(f)

In [ ]:
# lst_acc = [accuracy[i].tolist()[0] for i in range(len(accuracy))]
# epochs = [i+1 for i in range(num_epochs)]
# f, ax = plt.subplots(1)
# ax.plot(epochs, lst_acc,marker = 'o')
# ax.set_ylim(bottom=0,top = 0.7)
# plt.title('Plot showing the variations in accuracy for training set in the Convolutional NN working on\
#  normalized images over 40 epochs \n')
# plt.xlabel('Epochs')
# plt.ylabel('Magnitude')
# plt.legend(['accuracy'])
# plt.show(f)

In [ ]:

# lst_loss = [loss_np[i].tolist()[0] for i in range(len(accuracy))]
# epochs = [i+1 for i in range(num_epochs)]
# f, ax = plt.subplots(1)
# ax.plot(epochs, lst_loss, 'r', marker = 'o',)
# ax.set_ylim(bottom=0)
# plt.title('Plot showing the variations in loss for training set in the Convolutional NN working on\
#  normalized images over 40 epochs \n')
# plt.xlabel('Epochs')
# plt.ylabel('Magnitude')
# plt.legend(['loss'])
# plt.show(f)